In [11]:
import numpy as np
from selenium import webdriver
from time import sleep
import random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import json
import re




In [7]:
def get_product(driver, link):
    driver.get(link)
    sleep(random.randint(5,10))
    result = []
    product_list = driver.find_elements(By.CSS_SELECTOR, '.non-pc')
    for products in product_list:
        product_name = products.find_element(By.CSS_SELECTOR, 'h3.zE.zF.qB').text.strip() if products.find_elements(By.CSS_SELECTOR, 'h3.zE.zF.qB') else None
        product_price = products.find_element(By.CSS_SELECTOR, 'div.z4.nW.e7.gC').text.strip() if products.find_elements(By.CSS_SELECTOR, 'div.z4.nW.e7.gC') else None
        product_price_number = int(re.sub(r'[^\d]', '', product_price)) if product_price else None
        product_store = products.find_element(By.CSS_SELECTOR, 'h5.zM.lc.qB.g').text.strip() if products.find_elements(By.CSS_SELECTOR, 'h5.zM.lc.qB.g') else None
        product_image = products.find_element(By.CSS_SELECTOR, 'img.z9.z7').get_attribute('src') if products.find_elements(By.CSS_SELECTOR, 'img.z9.z7') else None
        on_attribute = products.get_attribute('on') if products else None
        on_attribute = products.get_attribute('on') if products else None
        product_link = None
        final_link = None
        if on_attribute:
            match = re.search(r"url='([^']+)'", on_attribute)
            product_link = match.group(1) if match else None
            if product_link:
                full_link = f"https://iprice.vn{product_link}"
                driver.get(full_link)
                sleep(random.randint(3, 5))
                final_link = driver.current_url  
                driver.back()
                sleep(random.randint(3, 5))  
        result.append({
            'product_name': product_name,
            'product_price': product_price_number,
            'product_store': product_store,
            'product_image': product_image,
            'product_link': final_link,
        })
    return result

In [8]:
def get_category(url, url_1):
    service = Service('chromedriver.exe')
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    sleep(random.randint(10, 15))
    result = []
    try:
        categories = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'ul > li.pointer[data-category]'))
        )
        for category in categories:
            sub_category_list = category.find_elements(By.CSS_SELECTOR, 'ul > li.pointer[data-category]')
            if sub_category_list:
                try:
                    if category.find_elements(By.CSS_SELECTOR, 'a[href] > span'):
                        main_category = category.find_element(By.CSS_SELECTOR, 'a[href] > span').text.strip()
                        link_main = category.find_element(By.CSS_SELECTOR, 'a[href]').get_attribute('href')
                    elif category.find_elements(By.CSS_SELECTOR, '> span[on] > span'):
                        main_category = category.find_element(By.CSS_SELECTOR, '> span[on] > span').text.strip()
                        on_attribute = category.find_element(By.CSS_SELECTOR, '> span[on]').get_attribute('on')
                        match = re.search(r"thor:\s*'([^']+)'", on_attribute)
                        if match:
                            link_main = f"{url_1}/{match.group(1).replace('|', '/')}/"
                        else:
                            link_main = None
                    else:
                        continue
                except Exception as e:
                    print(f"Error: {e}")
                    continue
                main_product = []
                if link_main:
                    main_product = get_product(driver, link_main)
                    driver.back()
                sub_result = []
                for sub_category in sub_category_list:
                    try:
                        if sub_category.find_elements(By.CSS_SELECTOR, 'a[href]'):
                            sub_category_element = sub_category.find_element(By.CSS_SELECTOR, 'a[href]')
                            sub_link = sub_category_element.get_attribute('href')
                            sub_products = get_product(driver, sub_link) if sub_link else []
                            driver.back()
                            sub_result.append({
                                "subCategory": sub_category_element.find_element(By.CSS_SELECTOR, 'span').text,
                                "products": sub_products,
                            })
                        elif sub_category.find_elements(By.CSS_SELECTOR, 'span[on]'):
                            sub_category_element = sub_category.find_element(By.CSS_SELECTOR, 'span[on]')
                            on_attribute = sub_category_element.get_attribute('on')
                            match = re.search(r"thor:\s*'([^']+)'", on_attribute)
                            if match:
                                sub_link = f"{url_1}/{match.group(1).replace('|', '/')}/"
                            else:
                                sub_link = None
                            sub_products = get_product(driver, sub_link) if sub_link else []
                            driver.back()
                            sub_result.append({
                                "subCategory": sub_category_element.find_element(By.CSS_SELECTOR, 'span').text,
                                "products": sub_products,
                            })
                    except Exception as e:
                        print(f"Error (subCategory): {e}")
                result.append({
                    'main_category': main_category,
                    'products': main_product,
                    'sub_categories': sub_result
                })
                break
    except Exception as e:
        print(f"Error: {e}")
    with open('categories_v1.json', 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)

    print("Dữ liệu đã được lưu vào file `categories_v1.json`.")
    return categories

In [9]:
url = 'https://iprice.vn/categories/'
url_1 = 'https://iprice.vn'
get_category(url, url_1)

Dữ liệu đã được lưu vào file `categories_v1.json`.


[<selenium.webdriver.remote.webelement.WebElement (session="c1e8d3a7e443259a05a58f341cce5fd8", element="f.95225DA3F0C6AA15076554A5F0E753EA.d.7B22BA3D5326A3DEC0B0F09D05ABE300.e.21")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c1e8d3a7e443259a05a58f341cce5fd8", element="f.95225DA3F0C6AA15076554A5F0E753EA.d.7B22BA3D5326A3DEC0B0F09D05ABE300.e.22")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c1e8d3a7e443259a05a58f341cce5fd8", element="f.95225DA3F0C6AA15076554A5F0E753EA.d.7B22BA3D5326A3DEC0B0F09D05ABE300.e.23")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c1e8d3a7e443259a05a58f341cce5fd8", element="f.95225DA3F0C6AA15076554A5F0E753EA.d.7B22BA3D5326A3DEC0B0F09D05ABE300.e.24")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c1e8d3a7e443259a05a58f341cce5fd8", element="f.95225DA3F0C6AA15076554A5F0E753EA.d.7B22BA3D5326A3DEC0B0F09D05ABE300.e.25")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c1e8d3a7e443259a05a58f341